## Ravish Chawla
### Topic Modeling with LDA and NMF algorithms on the ABC News Headlines Dataset
#### July 31, 2017
#### Minor changes for Topic Modeling Workshop at Northwestern University, August, 2019.

# Data imports

We import Pandas, numpy and scipy for data structures. We use gensim for LDA, and sklearn for NMF

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle

# Loading the data

We are using the ABC News headlines dataset. Some lines are badly formatted (very few), so we are skipping those.

In [ ]:
# Processing all the headlines takes a long time, so we'll work with a 5% 
# random sample instead.

# To use all the headlines, comment out the next line which loads the sample,
# and uncomment the subsequent line which loads the entire set of headlines.

data = pd.read_csv('data/abcnews-headlines-five-percent-sample.csv', error_bad_lines=False)
#data = pd.read_csv('data/abcnews-headlines-full.csv', error_bad_lines=False)


**We only need the Headlines_text column from the data**

In [ ]:
data_text = data[['headline_text']]

**Remove the stopwords.**

In [ ]:
data_text = data_text.astype('str')
stop_words = set(stopwords.words('english'))

In [ ]:
for idx in range(len(data_text)):
    
    #go through each word in each data_text row, remove stopwords, and set 
    #them on the index.
    data_text.iloc[idx]['headline_text'] = \
        [word for word in data_text.iloc[idx]['headline_text'].strip().split(' ') \
        if word not in stop_words and ( len(word) > 0 ) ]
    
    #print logs to monitor output
    if idx % 10000 == 0:
        sys.stdout.write('\rc = ' + str(idx) + ' / ' + str(len(data_text)))

sys.stdout.write('\rc = ' + str(idx + 1) + ' / ' + str(len(data_text)))
sys.stdout.write('  Completed tokenization and stopword removal.')

In [ ]:
#get the words as an array for lda input
train_headlines = [value[0] for value in data_text.iloc[0:].values]

In [ ]:
#number of topics we will extract: 10
num_topics = 10

# LDA

We will use the gensim library for LDA. First, we obtain a id-2-word dictionary. 
For each headline, we will use the dictionary to obtain a mapping of the word id 
to their word counts. The LDA model uses both of these mappings.

In [ ]:
id2word = gensim.corpora.Dictionary(train_headlines)

In [ ]:
corpus = [id2word.doc2bow(text) for text in train_headlines]

In [ ]:
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)

# Generating LDA topics

We will iterate over the number of topics, get the top words in each cluster and 
add them to a dataframe.

In [ ]:
def get_lda_topics(model, num_topics):
    word_dict = {}
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20)
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    return pd.DataFrame(word_dict)

In [ ]:
get_lda_topics(lda, num_topics)

# NMF

For NMF, we need to obtain a design matrix. To improve results, I am going to apply 
TfIdf transformation to the counts.

In [ ]:
#the count vectorizer needs string inputs, not array, so I join them with a space.
train_headlines_sentences = [' '.join(text) for text in train_headlines]

Now, we obtain a Counts design matrix, for which we use SKLearn’s CountVectorizer 
module.  The transformation will return a matrix of size (Documents x Features), 
where the value of a cell is going to be the number of times the feature (word) 
appears in that document.

To reduce the size of the matrix, to speed up computation, we will set the maximum 
feature size to 5000, which will take the top 5000 best features that can contribute 
to our model.

In [ ]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
x_counts = vectorizer.fit_transform(train_headlines_sentences)

Next, we set a TfIdf Transformer, and transform the counts with the model.

In [ ]:
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)

And now we normalize the TfIdf values to unit length for each row.

In [ ]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

Obtain an NMF model, and fit it with the sentences.
We use a singular value decomposition to initialize the topic extraction 
rather than a random state.

In [ ]:

model = NMF(n_components=num_topics, init='nndsvd')

In [ ]:
# Fit the model.
model.fit(xtfidf_norm)

In [ ]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can 
    #print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map 
        #to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(model, 20)

**The following discussion applies to the topics produced by using the 5% sample of headlines.  If you are running the full set, the results may differ.**

The two tables above, in each section, show the results from LDA and NMF on both datasets. There is some coherence between the words in each clustering. For example, Topic #01 in LDA shows words associated with potentially violent incidents, such as “police”, “suicide”, and “dying”. Other topics show different patterns. 

On the other hand, comparing the results of LDA to NMF also shows that NMF performs better. Looking at Topic #01, we can see there are many first names clustered into the same category, along with the word “interview”. This type of headline is very common in news articles, with wording similar to “Interview with John Smith”, or “Interview with James C. on …”. 

We also see two topics related to violence. First, Topic #04 focuses on police related terms, such as “probe”, “missing”, “investigate”, “arrest”, and “body”. Second, Topic #02 focuses on assault terms, such as “murder”, “stabbing”, “guilty”, and “killed”. This is an interesting split between the topics because although the terms in each are very closely related, one focuses more on police-related activity, and the other more on criminal activity. Along with the first cluster which obtain first-names, the results show that NMF (using TfIdf) performs much better than LDA on this set of texts.